# Init

In [2]:
from pysmt.shortcuts import Not, And, Symbol, TRUE
from pysmt.shortcuts import GE, Plus, Times, Real
from pysmt.shortcuts import Solver
from pysmt.typing import REAL, INT

from sage.all import Polyhedron

import numpy as np

In [3]:
def inv(l):
    return [-i for i in list(l)]


# in: P, polyhedron in V-representation or H-representation
# out: A, b such that P = {x | Ax >= b}
def H_to_array(P):
    H =  P.Hrepresentation()
    A = []
    b = []
    for ieq in H:
        if ieq.is_inequality():
            A.append(list(ieq.A()))
            b.append(ieq.b())
        elif ieq.is_equation():
            inv(list(ieq.A()))
            A.append(list(ieq.A()))
            A.append(inv(list(ieq.A())))
            b.append(ieq.b())
            b.append(-ieq.b())
    return (np.array(A),np.array(b))


# in: A, b such that P = {x | Ax >= b} (no equality)
# out: P, polyhedron in V-representation or H-representation
def array_to_H(A,b):
    assert(len(A)==len(b))
    return Polyhedron(ieqs=[[b[i]] + list(A[i]) for i in range(len(A))])

# in: A, b such that P = {x | Ax >= b} (no equality)
# out: f a formula computable by PySMT
def array_to_pysmt(A,b):
    pass



exs = [Polyhedron(vertices=[[0.5, 0], [0, 0.5]]),
       Polyhedron(vertices=[[0, 0,0], [0, 0,1], [0, 1,0], [0, 1,1], [1, 0,0], [1, 0,1], [1, 1,0], [1, 1,1]])
      ]
for ex in exs:
    P = H_to_array(ex)
    print("init",ex.Hrepresentation())
    print("final",array_to_H(P[0],P[1]).Hrepresentation())
    print()


init (An inequality (-2.0, 0.0) x + 1.0 >= 0, An inequality (1.0, 0.0) x + 0.0 >= 0, An equation (1.0, 1.0) x - 0.5 == 0)
final (An inequality (-2.0, 0.0) x + 1.0 >= 0, An inequality (1.0, 0.0) x + 0.0 >= 0, An equation (1.0, 1.0) x - 0.5 == 0)

init (An inequality (0, 0, -1) x + 1 >= 0, An inequality (0, -1, 0) x + 1 >= 0, An inequality (-1, 0, 0) x + 1 >= 0, An inequality (1, 0, 0) x + 0 >= 0, An inequality (0, 0, 1) x + 0 >= 0, An inequality (0, 1, 0) x + 0 >= 0)
final (An inequality (-1, 0, 0) x + 1 >= 0, An inequality (0, -1, 0) x + 1 >= 0, An inequality (0, 0, -1) x + 1 >= 0, An inequality (1, 0, 0) x + 0 >= 0, An inequality (0, 0, 1) x + 0 >= 0, An inequality (0, 1, 0) x + 0 >= 0)



## DDNF

In [4]:
# returns the same polyhedr
def add_var(P):
    return Polyhedron([ieq+[0] for ieq in P.Hrepresentation()])

#in: P1,P2 , polyhedra
#    res a list of disjoint polyhedra with P1 included in P2
# out: a point p in P2\(P1 U res) or False if empty
def find_point(P1,P2,r):
    A1,b1 = H_to_array(P1)
    A2,b2 = H_to_array(P2)
    
    res = []
    for i in range(len(r)):
        res.append(H_to_array(r[i]))


    assert(len(A1[0])==len(A2[0]))
    if res != []:
        for i in range(-len(A1[0])+len(res[0][0])):
            A1[i].append(0)
            A2[i].append(0)
        print("A1",A1[0],"A2",res[0][0])
    assert(r==[] or len(A1[0])==len(res[0][0]))

    n = len(A1[0])
    variables = [Symbol(str(s),REAL) for s in range(n)]

    formula = TRUE

    for i in range(len(A1)):
        la=A1[i]
        new_v = []
        for j in range(len(variables)):
            v = variables[j]
            new_v.append(Times(v,Real(float(la[j]))))
        formula = GE(Plus(new_v),Real(float(b1[i])))

    for (A,b) in [(A2,b2)] + res:
        for i in range(len(A)):
            la = A[i]
            new_v = []
            for j in range(len(variables)):
                v = variables[j]
                new_v.append(Times(v,Real(float(la[j]))))
            formula = And(formula,Not(GE(Plus(new_v),Real(float(b[i])))))
            
    model = []

    with Solver() as solver:
        solver.add_assertion(formula)
        if solver.solve():
            for v in variables:
                model.append(solver.get_value(v))
            return model
        else:
            return None
    



# in: P a polyhedron {Ax>=b} (no equality constraint)
#     pos a list number of rows
# out: P = {A'x>=b'} & {A''x - delta >=b''} for the rows in pos
# def strict_to_non_strict(P,pos):
#     ieqs = []
#     arr = H_to_array(P)
#     P = array_to_H(arr[0],arr[1])
#     H = P.Hrepresentation()
#     for i in range(len(H)):
#         ieq = H[i]
#         print(ieq)
#         assert(ieq.is_inequality())
#         if i in pos:
#             ieqs.append(list(ieq).append(-1))
#         else:
#             ieqs.append(ieq)
#     return Polyhedron(ieqs=ieqs)

# return the smallest polyhedron definable with the inequalities of P1 and P2 containing p
def new_Poly(P1,P2,p):
    p = np.array(p)
    ieqs = []
    n = 0
    for P in [P1,P2]:
        for ieq in P.Hrepresentation():
            la = np.array(list(ieq.A()))
            lb = ieq.b()
            if ieq.is_inequality():
                if np.dot(la.T,p) < lb:
                    ieqs.append(list(ieq))
                else:
                    e = inv(ieq)
                    e[-1] +=1
                    ieqs.append(e) #strict
            elif ieq.is_equation():
                if np.dot(la.T,p) >= lb:
                    ieqs.append(list(ieq))
                else:
                    e = inv(ieq)
                    e[-1] +=1
                    ieqs.append(e) #strict
                n+=1
                if np.dot(la.T,p) <= lb:
                    ieqs.append(inv(ieq))
                else:
                    e = list(ieq)
                    e[-1] -=1
                    ieqs.append(e) #strict
        n+=1
    return Polyhedron(ieqs=ieqs)

In [5]:
# in: P1,P2, polyhedra in H/V-representation
# out: res, list of disjoints polyhedra with first element P1 
def ddnf(P1,P2):
    res = [] # list of disjoints polyhedra
    P1 = add_var(P1)
    P2 = add_var(P2)
    p = find_point(P1,P2,res)
    while p != None:
        print(P1,P2)
        print(p)
        nP = new_Poly(P1,P2,p)
        res.append(nP)
        p = find_point(P1,P2,res)
    return [P1] + res


exs = [Polyhedron(vertices=[[0.5, 0,0], [0, 0.5,0]]),
       Polyhedron(vertices=[[0, 0,0], [0, 0,1], [0, 1,0], [0, 1,1], [1, 0,0], [1, 0,1], [1, 1,0], [1, 1,1]])
      ]

ddnf(exs[1],exs[0])


[A 3-dimensional polyhedron in ZZ^5 defined as the convex hull of 6 vertices]